# Извлечение геометрических признаков из 3D модели

### Цель: Создать 1D сигналы инвариантных (устойчивых) к трансформации

#### Примерный пайплайн для реализации

1) Спектральные сигнатуры (глобальные, инвариантные к перестановкам вершин):
   - Лапласиан (LB) спектр: первые k собственных значений/плотность спектра.
   - HKS/WKS: многошкальные подписи, можно усреднять по поверхности и бинировать в 1D-гистограммы (по времени/энергии).
2) Радиальные/угловые сигнатуры (геометрия в сферических координатах):
   - Radial Distance Function (RDF): из центра масс по K направлениям (Fibonacci sphere) измерять расстояние до поверхности → 1D-вектор длины K. Для вращательной устойчивости — PCA-выравнивание или сильные аугментации поворота; для круговой периодики — circular padding в Conv1d.
   - Spherical power spectrum: мощность по сферическим гармоникам (инвариант к вращению).
3) Многошкальные локальные дескрипторы → глобальные 1D-гистограммы:
   - FPFH/SHOT/curvatures (k1,k2, shape index, curvedness, нормали, локальная плотность): строим гистограммы по нескольким радиусам, получаем набор 1D-векторов.
4) Графовые/топологические сигнатуры:
   - Степени вершин, распределение длин рёбер/углов, число компонент, Эйлерова характеристика по изолиниям/уровням — как каналы 1D-гистограмм.
5) B-Rep признаки:
   - Больше типов поверхностей (NURBS со сложностью), статистики кривизн по дискретизации поверхности, отношения площадей типов, распределение параметров (радиусы, углы), граф смежности граней (степени/циклы) → свести в гистограммы/спектры.

1) загрузите датасет и выведите информацию 

In [42]:
import notebook_setup
from src.dataset import DatasetIO
from src.config import INTERIM_DATA_DIR

pkl_file = INTERIM_DATA_DIR / "dataset_metadata.pkl"

dataset = DatasetIO.load_dataset_pickle(pkl_file)

print(dataset)


2025-08-11 17:08:30.270 | INFO     | src.dataset:load_dataset_pickle:332 - Загрузка датасета из /home/developer/workspace/projects/3d_recognition_analisis/data/interim/dataset_metadata.pkl
2025-08-11 17:08:30.282 | SUCCESS  | src.dataset:load_dataset_pickle:337 - Датасет загружен из /home/developer/workspace/projects/3d_recognition_analisis/data/interim/dataset_metadata.pkl
[DataModel(model_id='44. Extractor Pin-06', model_path='/home/developer/workspace/projects/3d_recognition_analisis/data/raw/3D/44. Extractor Pin/44. Extractor Pin-06.prt.stp', detail_type='44. Extractor Pin', image_paths=[ImageData(image_id='44. Extractor Pin-06_left', image_path='/home/developer/workspace/projects/3d_recognition_analisis/data/raw/2D/44. Extractor Pin/44. Extractor Pin-06/44. Extractor Pin-06_Left.jpg', model_type='left'), ImageData(image_id='44. Extractor Pin-06_right', image_path='/home/developer/workspace/projects/3d_recognition_analisis/data/raw/2D/44. Extractor Pin/44. Extractor Pin-06/44. Extr

2) создаем триангуляцию модели. Результатом мы должны получить вершины и грани.

In [43]:
# STEP -> Trimesh: триангуляция через pythonocc, сборка в trimesh

import numpy as np

from OCC.Core.STEPControl import STEPControl_Reader
from OCC.Core.IFSelect import IFSelect_RetDone
from OCC.Core.BRepMesh import BRepMesh_IncrementalMesh
from OCC.Core.BRep import BRep_Tool
from OCC.Core.TopLoc import TopLoc_Location
from OCC.Extend.TopologyUtils import TopologyExplorer

from OCC.Core.BRep import BRep_Builder
from OCC.Core.TopoDS import TopoDS_Compound, topods
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_FACE
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge

def load_step_shape(step_path: str):
    reader = STEPControl_Reader()
    status = reader.ReadFile(step_path)
    if status != IFSelect_RetDone:
        print(f"Не удалось прочитать STEP: {step_path} (status={status})")
        return None
    reader.TransferRoots()
    shape = reader.OneShape()
    return shape

def mesh_shape(shape, lin_deflection=0.05, ang_deflection=0.5, is_relative=True, parallel=True):
    BRepMesh_IncrementalMesh(shape, lin_deflection, is_relative, ang_deflection, parallel)

def collect_triangulation_test(shape):
    """Тестовая функция для сбора триангуляции из формы."""
    # https://github.com/tpaviot/pythonocc-demos/blob/master/examples/core_simple_mesh.py
    verts_chunks = []
    faces_chunks = []
    v_off = 0

    ex = TopExp_Explorer(shape, TopAbs_FACE) # type: ignore
    while ex.More():
        face = topods.Face(ex.Current())
        location = TopLoc_Location()
        # получим триангуляцию 
        facing = BRep_Tool.Triangulation(face, location)
        
        if facing is None:
            continue
        trsf = location.Transformation()
        # получим вершины
        nb_nodes = facing.NbNodes()
        cur_verts = np.empty((nb_nodes, 3), dtype=np.float64)
        for i in range(1, nb_nodes + 1):
            p = facing.Node(i).Transformed(trsf)
            cur_verts[i - 1] = [p.X(), p.Y(), p.Z()]
        verts_chunks.append(cur_verts)

        # получим грани
        nb_tris = facing.NbTriangles()
        cur_faces = np.empty((nb_tris, 3), dtype=np.int64)
        for i in range(1, nb_tris + 1):
            t = facing.Triangle(i)
            i1, i2, i3 = t.Get()
            cur_faces[i - 1] = [v_off + i1 - 1, v_off + i2 - 1, v_off + i3 - 1]
        faces_chunks.append(cur_faces)
        ex.Next()
        v_off += nb_nodes
    
    V = np.vstack(verts_chunks)
    F = np.vstack(faces_chunks)
    return V, F

def collect_triangulation(shape):
    verts_chunks = []
    faces_chunks = []
    v_off = 0

    topo = TopologyExplorer(shape)
    for face in topo.faces():
        loc = TopLoc_Location()
        triangulation = BRep_Tool.Triangulation(face, loc)
        if triangulation is None:
            continue

        trsf = loc.Transformation()

        nb_nodes = triangulation.NbNodes()
        cur_verts = np.empty((nb_nodes, 3), dtype=np.float64)
        for i in range(1, nb_nodes + 1):
            p = triangulation.Node(i).Transformed(trsf)
            cur_verts[i - 1] = [p.X(), p.Y(), p.Z()]
        verts_chunks.append(cur_verts)

        nb_tris = triangulation.NbTriangles()
        cur_faces = np.empty((nb_tris, 3), dtype=np.int64)
        for i in range(1, nb_tris + 1):
            t = triangulation.Triangle(i)
            i1, i2, i3 = t.Get()
            cur_faces[i - 1] = [v_off + i1 - 1, v_off + i2 - 1, v_off + i3 - 1]
        faces_chunks.append(cur_faces)

        v_off += nb_nodes

    if not verts_chunks or not faces_chunks:
        return np.empty((0, 3), dtype=np.float64), np.empty((0, 3), dtype=np.int64)

    V = np.vstack(verts_chunks)
    F = np.vstack(faces_chunks)
    return V, F # вершины и грани


# ----- Использование -----
model_path = dataset[0].model_path
shape = load_step_shape(model_path)

# более грубая и стабильная триангуляция для диагностики
mesh_shape(shape, lin_deflection=0.02, ang_deflection=0.785, is_relative=True, parallel=False)

V, F = collect_triangulation_test(shape)

# V, F = collect_triangulation(shape)

print(f"Vertices: {V.shape}, Faces: {F.shape}")
print("First 5 vertices:\n", V[:5])
print("First 5 faces:\n", F[:5])

# from OCC.Display.WebGl.jupyter_renderer import JupyterRenderer
# from OCC.Display.WebGl.x3dom_renderer import X3DomRenderer
# renderer = X3DomRenderer()
# renderer.DisplayShape(shape)
# renderer.render()

Vertices: (228, 3), Faces: (208, 3)
First 5 vertices:
 [[0.         0.1        1.        ]
 [0.36124167 0.1        0.93247223]
 [0.67369564 0.1        0.73900892]
 [0.89516329 0.1        0.44573836]
 [0.99573418 0.1        0.09226836]]
First 5 faces:
 [[36 37 38]
 [36 38 39]
 [19 17 18]
 [35 39 40]
 [35 36 39]]


3) Локальные гистограммы по триангуляции базовые признаки: распределение длин рёбер, площадей треугольников, диэдральных углов, степени вершин и Эйлерову характеристику.

In [44]:
import numpy as np

def mesh_edge_data(V, F):
    # уникальные рёбра
    E = np.concatenate([F[:, [0,1]], F[:, [1,2]], F[:, [2,0]]], axis=0)
    E.sort(axis=1)
    E = np.unique(E, axis=0)
    L = np.linalg.norm(V[E[:,0]] - V[E[:,1]], axis=1)
    return E, L

def mesh_face_areas(V, F):
    v0 = V[F[:,0]]; v1 = V[F[:,1]]; v2 = V[F[:,2]]
    A = 0.5 * np.linalg.norm(np.cross(v1 - v0, v2 - v0), axis=1)
    return A

def mesh_face_normals(V, F):
    v0 = V[F[:,0]]; v1 = V[F[:,1]]; v2 = V[F[:,2]]
    n = np.cross(v1 - v0, v2 - v0)
    n_norm = np.linalg.norm(n, axis=1, keepdims=True) + 1e-12
    return n / n_norm

def mesh_dihedral_angles(V, F):
    # соседство по рёбрам
    E = np.concatenate([F[:, [0,1]], F[:, [1,2]], F[:, [2,0]]], axis=0)
    E_sorted = np.sort(E, axis=1)
    tri_idx = np.repeat(np.arange(F.shape[0]), 3)
    from collections import defaultdict
    edge2tris = defaultdict(list)
    for e, t in zip(map(tuple, E_sorted), tri_idx):
        edge2tris[e].append(t)

    N = mesh_face_normals(V, F)
    ang = []
    for tris in edge2tris.values():
        if len(tris) == 2:
            i, j = tris
            c = np.clip((N[i] * N[j]).sum(), -1.0, 1.0)
            ang.append(np.arccos(c))
    return np.array(ang, dtype=np.float64)

def mesh_vertex_degrees(F, Vn):
    # степень вершины как кол-во инцидентных рёбер
    deg = np.zeros(Vn, dtype=np.int32)
    E = np.concatenate([F[:, [0,1]], F[:, [1,2]], F[:, [2,0]]], axis=0)
    for a, b in E:
        deg[a] += 1
        deg[b] += 1
    return deg

def hist_norm(x, bins=64, rng=None, log=False):
    if x.size == 0:
        return np.zeros(bins, dtype=np.float32)
    vals = x
    if log:
        vals = np.log10(np.clip(vals, 1e-12, None))
    H, _ = np.histogram(vals, bins=bins, range=rng, density=True)
    return H.astype(np.float32)

# Использование: предполагаем, что V, F уже получены вашей collect_triangulation
E, L = mesh_edge_data(V, F)
A = mesh_face_areas(V, F)
D = mesh_dihedral_angles(V, F)
deg = mesh_vertex_degrees(F, V.shape[0])

h_edge = hist_norm(L, bins=64, log=True)
h_area = hist_norm(A, bins=64, log=True)
h_dih  = hist_norm(D, bins=64, rng=(0, np.pi))
h_deg  = hist_norm(deg, bins=32, rng=(0, deg.max() if deg.size else 1))

print("Dims:", h_edge.shape, h_area.shape, h_dih.shape, h_deg.shape)

Dims: (64,) (64,) (64,) (32,)


Вычисляется RDF — сигнатура, описывающая геометрию относительно центра масс. Лучи испускаются по равномерно распределенным направлениям (сфера Фибоначчи), и измеряется расстояние до пересечения с поверхностью

In [45]:
import numpy as np

def fibonacci_sphere(n: int, seed: int = 0):
    i = np.arange(n, dtype=np.float64)
    phi = (1 + 5 ** 0.5) / 2
    theta = 2 * np.pi * i / phi
    z = 1 - (2 * i + 1) / n
    r = np.sqrt(np.maximum(0.0, 1 - z * z))
    x = r * np.cos(theta); y = r * np.sin(theta)
    dirs = np.stack([x, y, z], axis=1)
    return dirs

def ray_triangle_intersections(orig, dirv, V, F):
    # Возвращает минимальную положительную дистанцию до пересечения с любым треугольником, либо np.inf
    v0 = V[F[:,0]]; v1 = V[F[:,1]]; v2 = V[F[:,2]]
    eps = 1e-9

    e1 = v1 - v0
    e2 = v2 - v0
    pvec = np.cross(dirv, e2)
    det = (e1 * pvec).sum(axis=1)
    mask = np.abs(det) > eps
    inv_det = np.zeros_like(det)
    inv_det[mask] = 1.0 / det[mask]

    tvec = orig - v0
    u = (tvec * pvec).sum(axis=1) * inv_det
    qvec = np.cross(tvec, e1)
    v = (dirv * qvec).sum(axis=1) * inv_det
    t = (e2 * qvec).sum(axis=1) * inv_det

    cond = (mask) & (u >= 0) & (v >= 0) & (u + v <= 1) & (t > eps)
    t_valid = np.where(cond, t, np.inf)
    return t_valid.min()

def compute_rdf(V, F, K=256, seed=0):
    c = V.mean(axis=0)
    rmax = np.linalg.norm(V - c, axis=1).max() + 1e-9
    dirs = fibonacci_sphere(K, seed)
    # нормируем направления на всякий случай
    dirs = dirs / (np.linalg.norm(dirs, axis=1, keepdims=True) + 1e-12)
    dists = np.empty(K, dtype=np.float64)
    for i, d in enumerate(dirs):
        dists[i] = ray_triangle_intersections(c, d, V, F)
        if not np.isfinite(dists[i]):
            dists[i] = rmax
    return (dists / rmax).astype(np.float32)

rdf = compute_rdf(V, F, K=256, seed=0)
print("RDF dim:", rdf.shape)

RDF dim: (256,)


 B‑Rep признаки (типы поверхностей и площади) считаем типы поверхностей граней и площади. Анализируется исходная B-Rep структура модели. Вычисляются суммарные площади для каждого типа поверхности (плоскость, цилиндр, NURBS и т.д.) и нормализуются, формируя вектор признаков
 

In [46]:
from OCC.Extend.TopologyUtils import TopologyExplorer
from OCC.Core.BRepAdaptor import BRepAdaptor_Surface
from OCC.Core.GeomAbs import (
    GeomAbs_Plane, GeomAbs_Cylinder, GeomAbs_Cone, GeomAbs_Sphere, GeomAbs_Torus,
    GeomAbs_BezierSurface, GeomAbs_BSplineSurface, GeomAbs_SurfaceOfRevolution,
    GeomAbs_SurfaceOfExtrusion, GeomAbs_OtherSurface
)
from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRepGProp import brepgprop

def face_area(face):
    props = GProp_GProps()
    brepgprop.SurfaceProperties(face, props)
    return props.Mass()  # для поверхности масса=площадь

def brep_surface_type_hist(shape):
    topo = TopologyExplorer(shape)
    type2area = {
        GeomAbs_Plane: 0.0,
        GeomAbs_Cylinder: 0.0,
        GeomAbs_Cone: 0.0,
        GeomAbs_Sphere: 0.0,
        GeomAbs_Torus: 0.0,
        GeomAbs_BezierSurface: 0.0,
        GeomAbs_BSplineSurface: 0.0,
        GeomAbs_SurfaceOfRevolution: 0.0,
        GeomAbs_SurfaceOfExtrusion: 0.0,
        GeomAbs_OtherSurface: 0.0
    }
    total_area = 0.0
    for face in topo.faces():
        surf = BRepAdaptor_Surface(face, True)
        st = surf.GetType()
        A = face_area(face)
        total_area += A
        if st in type2area:
            type2area[st] += A
        else:
            type2area[GeomAbs_OtherSurface] += A
    if total_area <= 0:
        return np.zeros(len(type2area), dtype=np.float32)
    # нормируем площади по типам
    keys = [GeomAbs_Plane, GeomAbs_Cylinder, GeomAbs_Cone, GeomAbs_Sphere, GeomAbs_Torus,
            GeomAbs_BezierSurface, GeomAbs_BSplineSurface, GeomAbs_SurfaceOfRevolution,
            GeomAbs_SurfaceOfExtrusion, GeomAbs_OtherSurface]
    vec = np.array([type2area[k] / total_area for k in keys], dtype=np.float32)
    return vec

brep_vec = brep_surface_type_hist(shape)
print("B-Rep type vector dim:", brep_vec.shape)

B-Rep type vector dim: (10,)


In [47]:
feature_vec = np.concatenate([
    rdf,            # K (например, 256)
    h_edge,         # 64
    h_area,         # 64
    h_dih,          # 64
    h_deg,          # 32
    brep_vec        # 10 типов поверхностей
]).astype(np.float32)

print("Total feature dim:", feature_vec.shape)

Total feature dim: (490,)


Спектральные сигнатуры: котангенсный Лапласиан + LB спектр + HKS/WKS

Решается задача на собственные значения для получения спектра (собственных значений evals) и собственных функций (evecs) оператора. Эти признаки являются инвариантными к изометриям.

In [48]:
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as spla
import scipy.sparse.csgraph as csgraph
from scipy.sparse.linalg import aslinearoperator

def total_area(V, F):
    v0, v1, v2 = V[F[:,0]], V[F[:,1]], V[F[:,2]]
    A = 0.5 * np.linalg.norm(np.cross(v1 - v0, v2 - v0), axis=1)
    return float(A.sum())

def clean_mesh_for_spectrum(V, F, area_eps=1e-14):
    if F.size == 0:
        return V, F
    v0, v1, v2 = V[F[:,0]], V[F[:,1]], V[F[:,2]]
    area2 = np.linalg.norm(np.cross(v1 - v0, v2 - v0), axis=1)
    mask = area2 > area_eps
    F = F[mask]
    if F.size == 0:
        return V[:0], F
    used = np.unique(F.ravel())
    remap = -np.ones(V.shape[0], dtype=np.int64)
    remap[used] = np.arange(used.size, dtype=np.int64)
    Vc = V[used]
    Fc = remap[F]
    return Vc, Fc

def build_laplacian_cotan(V, F):
    n = V.shape[0]
    i, j, k = F[:,0], F[:,1], F[:,2]
    vi, vj, vk = V[i], V[j], V[k]

    # удвоенные площади
    nrm = np.cross(vj - vi, vk - vi)
    area2 = np.linalg.norm(nrm, axis=1) + 1e-15

    # котангенсы углов при i, j, k
    cot_i = ((vj - vi) * (vk - vi)).sum(axis=1) / area2
    cot_j = ((vi - vj) * (vk - vj)).sum(axis=1) / area2
    cot_k = ((vi - vk) * (vj - vk)).sum(axis=1) / area2

    w_ij = 0.5 * cot_k
    w_jk = 0.5 * cot_i
    w_ki = 0.5 * cot_j

    rows = np.concatenate([i, j, j, k, k, i, i, j, k])
    cols = np.concatenate([j, i, k, j, i, k, i, j, k])
    data = np.concatenate([
        -w_ij, -w_ij,
        -w_jk, -w_jk,
        -w_ki, -w_ki,
        (w_ij + w_ki),
        (w_ij + w_jk),
        (w_jk + w_ki)
    ])
    L = sp.coo_matrix((data, (rows, cols)), shape=(n, n)).tocsr()

    # масс-матрица (барицентрическая)
    tri_area = 0.5 * (area2 - 1e-15)
    M_diag = np.zeros(n, dtype=np.float64)
    np.add.at(M_diag, i, tri_area / 3.0)
    np.add.at(M_diag, j, tri_area / 3.0)
    np.add.at(M_diag, k, tri_area / 3.0)
    M_diag = np.maximum(M_diag, 1e-15)
    M = sp.diags(M_diag)
    return L, M, M_diag

def keep_largest_component(V, F):
    if F.size == 0:
        return V, F
    n = V.shape[0]
    E = np.concatenate([F[:, [0,1]], F[:, [1,2]], F[:, [2,0]]], axis=0)
    E.sort(axis=1); E = np.unique(E, axis=0)
    rows = np.concatenate([E[:,0], E[:,1]])
    cols = np.concatenate([E[:,1], E[:,0]])
    data = np.ones(rows.shape[0], dtype=np.int8)
    G = sp.csr_matrix((data, (rows, cols)), shape=(n, n))
    ncomp, labels = csgraph.connected_components(G, directed=False)
    if ncomp <= 1:
        return V, F
    largest = np.argmax(np.bincount(labels))
    mask_v = labels == largest
    idx_old = np.where(mask_v)[0]
    remap = -np.ones(n, dtype=np.int64); remap[idx_old] = np.arange(idx_old.size, dtype=np.int64)
    Fm = remap[F]; Fm = Fm[(Fm >= 0).all(axis=1)]
    return V[idx_old], Fm

def compute_lbo_spectrum(V, F, k=32, scale_invariant=True):
    # чистим и берём крупнейшую компоненту
    Vc, Fc = clean_mesh_for_spectrum(V, F)
    Vc, Fc = keep_largest_component(Vc, Fc)
    if Vc.shape[0] < 3 or Fc.shape[0] < 1:
        raise RuntimeError("Недостаточно данных для спектра после чистки.")

    Vn = Vc
    if scale_invariant:
        A = total_area(Vc, Fc)
        if A > 0:
            Vn = Vc / np.sqrt(A)

    L, M, M_diag = build_laplacian_cotan(Vn, Fc)
    n = Vn.shape[0]
    k = min(k, max(1, n - 2))
    D = sp.diags(1.0 / np.sqrt(M_diag))
    Aop = (D @ L) @ D  # SPD, имеет нулевой корень

    d = np.asarray(Aop.diagonal()).ravel()
    d = np.maximum(d, 1e-12)
    Mop = aslinearoperator(sp.diags(1.0 / d))
        

    X0 = np.random.randn(n, min(k + 6, n - 1))
    ones = np.ones((n, 1))
    X0 = X0 - ones @ ((ones.T @ X0) / n)
    
    w, Uy = spla.lobpcg(Aop, X0, M=Mop, largest=False, tol=1e-4, maxiter=200)
    ok = np.isfinite(w); w, Uy = w[ok], Uy[:, ok]
    order = np.argsort(w); evals = w[order][:k]; Uy = Uy[:, order][:, :k]

    # отброс нулевого корня и возврат M-ортонормированных векторов: x = M^{-1/2} y
    pos = evals > 1e-10
    evals_k = evals[pos]
    evecs_k = (D @ Uy[:, pos]).astype(np.float64)
    return evals_k, evecs_k


k_spec = 16
evals, evecs = compute_lbo_spectrum(V, F, k=k_spec, scale_invariant=True)
print("LB spectrum:", evals.shape, evecs.shape)

LB spectrum: (15,) (76, 15)


/tmp/ipykernel_40350/77687951.py:116: UserWarning: The problem size 76 minus the constraints size 0 is too small relative to the block size 22. Using a dense eigensolver instead of LOBPCG iterations.No output of the history of the iterations.
  w, Uy = spla.lobpcg(Aop, X0, M=Mop, largest=False, tol=1e-4, maxiter=200)


In [49]:
# Создаем вектор для собственных значений фиксированного размера k_spec
# и заполняем его вычисленными значениями, дополняя нулями при необходимости.
padded_evals = np.zeros(k_spec, dtype=np.float32)
num_evals = min(len(evals), k_spec)
padded_evals[:num_evals] = evals[:num_evals]

# Объединяем все признаки в один вектор
feature_vec = np.concatenate([
    rdf,            # 256
    h_edge,         # 64
    h_area,         # 64
    h_dih,          # 64
    h_deg,          # 32
    brep_vec,       # 10
    padded_evals    # 16 (k_spec)
]).astype(np.float32)

print("Итоговый размер вектора признаков:", feature_vec.shape)

Итоговый размер вектора признаков: (506,)
